In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib
import re

In [2]:
import pandas as pd

# Read the Excel file
df = pd.read_excel('accident_df.xlsx')

# Save the DataFrame to a CSV file
# index=False prevents pandas from writing the DataFrame index as a column
df.to_csv('accident_df.csv', index=False)

In [3]:
df = pd.read_csv('accident_df.csv')

df.head()

,구분번호,발생년월,주야,시군구,사고내용,사망자수,중상자수,경상자수,부상신고자수,사고유형,...,기상상태,도로형태,가해운전자 차종,가해운전자 성별,가해운전자 연령대,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령대,피해운전자 상해정도
0,2022000001,2022년 1월,야간,서울특별시 중구,경상사고,0,0,1,0,차대차 - 기타,...,맑음,단일로 - 기타,승용,남,61-64세,상해없음,승용,남,65세 이상,경상
1,2022000002,2022년 1월,야간,서울특별시 송파구,중상사고,0,2,0,0,차대사람 - 횡단중,...,맑음,단일로 - 기타,승용,남,65세 이상,상해없음,보행자,여,13-20세,중상
2,2022000017,2022년 1월,야간,서울특별시 금천구,부상신고사고,0,0,0,1,차량단독 - 기타,...,맑음,단일로 - 기타,이륜,남,21-30세,부상신고,NaN,NaN,NaN,NaN
3,2022000018,2022년 1월,야간,서울특별시 강동구,경상사고,0,0,2,0,차대차 - 충돌,...,맑음,단일로 - 기타,승용,여,61-64세,상해없음,승용,남,61-64세,경상
4,2022000037,2022년 1월,주간,서울특별시 강남구,중상사고,0,1,0,0,차대사람 - 기타,...,맑음,교차로 - 교차로안,승용,남,61-64세,상해없음,보행자,여,61-64세,중상


In [4]:
df.describe()

,구분번호,사망자수,중상자수,경상자수,부상신고자수
count,1.009740e+05,100974.000000,100974.000000,100974.000000,100974.000000
mean,2.023096e+09,0.006071,0.229009,1.008170,0.100491
std,8.175255e+05,0.083812,0.469720,0.912991,0.360786
min,2.022000e+09,0.000000,0.000000,0.000000,0.000000
25%,2.022148e+09,0.000000,0.000000,1.000000,0.000000
50%,2.023100e+09,0.000000,0.000000,1.000000,0.000000
75%,2.024048e+09,0.000000,0.000000,1.000000,0.000000
max,2.024196e+09,9.000000,8.000000,34.000000,20.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100974 entries, 0 to 100973
Data columns (total 22 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   구분번호        100974 non-null  int64 
 1   발생년월        100974 non-null  object
 2   주야          100974 non-null  object
 3   시군구         100974 non-null  object
 4   사고내용        100974 non-null  object
 5   사망자수        100974 non-null  int64 
 6   중상자수        100974 non-null  int64 
 7   경상자수        100974 non-null  int64 
 8   부상신고자수      100974 non-null  int64 
 9   사고유형        100974 non-null  object
 10  법규위반        100974 non-null  object
 11  노면상태        100974 non-null  object
 12  기상상태        100974 non-null  object
 13  도로형태        100974 non-null  object
 14  가해운전자 차종    100974 non-null  object
 15  가해운전자 성별    100974 non-null  object
 16  가해운전자 연령대   100974 non-null  object
 17  가해운전자 상해정도  100974 non-null  object
 18  피해운전자 차종    97865 non-null   object
 19  피해운전자 성별    97865 non-n

도로명주소 확인, 사고일시 object, 연령 object, 피해운전자 차종 object

In [6]:
import pandas as pd
import sys

def preprocess_data(input_file_path: str) -> (str, list, list):
    """
    사고 데이터 CSV 파일을 로드하여 Streamlit 시각화에 적합하도록 전처리합니다.

    전처리 내용:
    1. 날짜(발생년월) 컬럼을 'Year'와 'Month'로 분리
    2. 피해자 정보 결측치를 '해당없음'으로 대체
    3. '총 사상자수' 파생 변수 생성
    4. 불필요한 '구분번호', '발생년월' 컬럼 제거
    5. 전처리된 데이터를 새 CSV 파일로 저장

    Args:
        input_file_path (str): 원본 CSV 파일 경로

    Returns:
        tuple: (저장된 파일 경로, 분석 차원 리스트, 측정 지표 리스트)
               오류 발생 시 (None, [], []) 반환
    """
    try:
        # 1. 데이터 로드
        # 파일 경로에 사용자가 제공한 파일 이름을 사용합니다.
        df = pd.read_csv(input_file_path)
        print(f"'{input_file_path}' 파일 로드 성공. (총 {len(df)} 행)")
    except FileNotFoundError:
        print(f"오류: '{input_file_path}' 파일을 찾을 수 없습니다.")
        print("스크립트가 CSV 파일과 동일한 폴더에 있는지 확인하세요.")
        return None, [], []
    except Exception as e:
        print(f"파일 로드 중 오류 발생: {e}")
        return None, [], []

    # 2. 날짜 데이터 처리 ('발생년월' -> '발생일시'(datetime), 'Year', 'Month')
    try:
        # 'YYYY년 MM월' 형식을 datetime 객체로 변환
        df['발생일시'] = pd.to_datetime(df['발생년월'], format='%Y년 %m월')
        # 편의를 위해 'Year', 'Month' 컬럼도 생성
        df['Year'] = df['발생일시'].dt.year
        df['Month'] = df['발생일시'].dt.month
        print("날짜 데이터 '발생일시'(datetime), 'Year', 'Month' 컬럼 생성 완료.")
    except Exception as e:
        print(f"날짜 처리 중 오류: {e}. '발생년월' 컬럼 형식을('%Y년 %m월') 확인하세요.")
        # 오류가 발생해도 일단 계속 진행
        pass

    # 3. 결측치 처리 (피해운전자 정보)
    # 차량단독 사고의 경우 피해운전자 정보가 NaN이 됩니다.
    victim_cols = ['피해운전자 차종', '피해운전자 성별', '피해운전자 연령대', '피해운전자 상해정도']
    # df.fillna()를 사용하여 해당 컬럼들의 NaN 값을 '해당없음'으로 채웁니다.
    df[victim_cols] = df[victim_cols].fillna('해당없음')
    print("결측치(NaN) '해당없음'으로 처리 완료.")

    # 4. 새 측정 지표 생성 ('총 사상자수')
    metric_cols = ['사망자수', '중상자수', '경상자수', '부상신고자수']
    # 숫자형이 아닌 값이 있을 경우를 대비해 numeric으로 변환 (오류 시 0으로)
    for col in metric_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)
        
    df['총 사상자수'] = df[metric_cols].sum(axis=1)
    print("'총 사상자수' 컬럼 생성 완료.")

    # 5. 불필요한 컬럼 제거
    cols_to_drop = ['구분번호', '발생년월']
    # '발생일시' 등이 성공적으로 처리되었을 경우 원본 컬럼 제거
    if '발생일시' in df.columns:
         df = df.drop(columns=cols_to_drop, errors='ignore')
    else:
         df = df.drop(columns=['구분번호'], errors='ignore')
         
    print("불필요한 원본 컬럼 제거 완료.")

    # 6. Streamlit에서 사용할 변수 목록 정의
    
    # 측정 지표 (숫자형, 집계 대상)
    metrics = ['총 사상자수', '사망자수', '중상자수', '경상자수', '부상신고자수']
    
    # 분석 차원 (범주형, 그룹화 대상)
    # 원본 컬럼에서 지표와 제거된 컬럼을 제외한 나머지
    all_cols = df.columns.tolist()
    dimensions = [col for col in all_cols if col not in metrics]

    # 7. 결과 저장
    output_file_path = 'accident_df_preprocessed.csv'
    try:
        # 한글 깨짐 방지를 위해 'utf-8-sig' 인코딩 사용
        df.to_csv(output_file_path, index=False, encoding='utf-8-sig')
        print(f"\n전처리 완료! 결과가 '{output_file_path}' 파일로 저장되었습니다.")
        return output_file_path, dimensions, metrics
    except Exception as e:
        print(f"결과 파일 저장 중 오류 발생: {e}")
        return None, [], []

if __name__ == "__main__":
    # 사용자가 업로드한 파일 이름을 정확히 지정합니다.
    original_file = "accident_df.csv"
    
    saved_file, dims, mets = preprocess_data(original_file)
    
    if saved_file:
        print("\n--- Streamlit 앱에서 사용할 수 있는 변수 목록 ---")
        print("\n[분석 차원 (Dimension)] - (예: X축, 그룹, 범례로 사용)")
        print(dims)
        
        print("\n[측정 지표 (Metric)] - (예: Y축, 값으로 사용)")
        print(mets)
        
        print("\n예시: Streamlit 앱에서 이 목록을 사용하여 드롭다운을 만드세요.")

'accident_df.csv' 파일 로드 성공. (총 100974 행)
날짜 데이터 '발생일시'(datetime), 'Year', 'Month' 컬럼 생성 완료.
결측치(NaN) '해당없음'으로 처리 완료.
'총 사상자수' 컬럼 생성 완료.
불필요한 원본 컬럼 제거 완료.

전처리 완료! 결과가 'accident_df_preprocessed.csv' 파일로 저장되었습니다.

--- Streamlit 앱에서 사용할 수 있는 변수 목록 ---

[분석 차원 (Dimension)] - (예: X축, 그룹, 범례로 사용)
['주야', '시군구', '사고내용', '사고유형', '법규위반', '노면상태', '기상상태', '도로형태', '가해운전자 차종', '가해운전자 성별', '가해운전자 연령대', '가해운전자 상해정도', '피해운전자 차종', '피해운전자 성별', '피해운전자 연령대', '피해운전자 상해정도', '발생일시', 'Year', 'Month']

[측정 지표 (Metric)] - (예: Y축, 값으로 사용)
['총 사상자수', '사망자수', '중상자수', '경상자수', '부상신고자수']

예시: Streamlit 앱에서 이 목록을 사용하여 드롭다운을 만드세요.


In [7]:
df1 = pd.read_csv("accident_df_preprocessed.csv")

df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100974 entries, 0 to 100973
Data columns (total 24 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   주야          100974 non-null  object
 1   시군구         100974 non-null  object
 2   사고내용        100974 non-null  object
 3   사망자수        100974 non-null  int64 
 4   중상자수        100974 non-null  int64 
 5   경상자수        100974 non-null  int64 
 6   부상신고자수      100974 non-null  int64 
 7   사고유형        100974 non-null  object
 8   법규위반        100974 non-null  object
 9   노면상태        100974 non-null  object
 10  기상상태        100974 non-null  object
 11  도로형태        100974 non-null  object
 12  가해운전자 차종    100974 non-null  object
 13  가해운전자 성별    100974 non-null  object
 14  가해운전자 연령대   100974 non-null  object
 15  가해운전자 상해정도  100974 non-null  object
 16  피해운전자 차종    100974 non-null  object
 17  피해운전자 성별    100974 non-null  object
 18  피해운전자 연령대   100974 non-null  object
 19  피해운전자 상해정도  100974 non-